## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [46]:
from arcgis.gis import GIS
gis = GIS("home")

#### Now you are ready to start!

In [47]:

import arcgis
from arcgis.gis import GIS
import pandas as pd
import datetime
import getpass
from IPython.display import HTML

from arcgis import geocoding
from arcgis.features import Feature, FeatureSet
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [48]:

my_gis = GIS('home')

In [49]:

orders_csv = "/arcgis/home/orders.csv"

# Read the csv file and convert the data to feature set
orders_df = pd.read_csv(orders_csv)
orders_sdf = pd.DataFrame.spatial.from_df(orders_df, "Address")
orders_fset = orders_sdf.spatial.to_featureset()

In [50]:

orders_sdf.head(3)

,Address,SHAPE
0,"602 Murray Cir, Sausalito, CA 94965","{""x"": -122.47885242199999, ""y"": 37.83735920100..."
1,"340 Stockton St, San Francisco, CA 94108","{""x"": -122.40685153899994, ""y"": 37.78910429100..."
2,"3619 Balboa St, San Francisco, CA 94121","{""x"": -122.49772620499999, ""y"": 37.77567413500..."


In [51]:
routes_csv = "/arcgis/home/routes.csv"

# Read the csv file
routes_df = pd.read_csv(routes_csv, parse_dates=["EarliestStartTime", "LatestStartTime"], date_parser=pd.to_datetime)
routes_df["EarliestStartTime"] = routes_df["EarliestStartTime"].astype("int64") / 10 ** 6
routes_df["LatestStartTime"] = routes_df["LatestStartTime"].astype("int64") / 10 ** 6
routes_fset = arcgis.features.FeatureSet.from_dataframe(routes_df)

In [52]:
routes_df

,ObjectID,Name,StartDepotName,EndDepotName,EarliestStartTime,LatestStartTime,Capacities,CostPerUnitTime,MaxOrderCount,MaxTotalTime,AssignmentRule
0,1,Route1,Warehouse,Warehouse,-2.209133e+12,-2.209126e+12,4,20,4,150,1
1,2,Route2,Warehouse,Warehouse,-2.209133e+12,-2.209126e+12,4,20,4,150,1
2,3,Route3,Warehouse,Warehouse,-2.209133e+12,-2.209126e+12,4,20,4,150,1


In [53]:
depots_csv = "/arcgis/home/depots.csv"

# Read the csv file and convert the data to feature set
depots_df = pd.read_csv(depots_csv)
depots_sdf = pd.DataFrame.spatial.from_xy(depots_df, "Longitude", "Latitude")
depots_sdf = depots_sdf.drop(axis=1, labels=["Longitude", "Latitude"])
depots_fset = depots_sdf.spatial.to_featureset()
depots_sdf

,Name,SHAPE
0,Warehouse,"{""spatialReference"": {""wkid"": 4326}, ""x"": -122..."


In [54]:

# Create a map instance to visualize inputs in map
map_view_inputs = my_gis.map('San Fransisco, United States', zoomlevel=10)
map_view_inputs

MapView(layout=Layout(height='400px', width='100%'))

In [55]:
# Visualize order and depot locations with symbology
map_view_inputs.draw(orders_fset, symbol={"type": "esriSMS","style": "esriSMSCircle","color": [76,115,0,255],"size": 8})
map_view_inputs.draw(depots_fset, symbol={"type": "esriSMS","style": "esriSMSSquare","color": [255,115,0,255], "size": 10})

In [56]:
depot_geocoded_fs = geocoding.geocode("2-98 Pier 1, San Francisco, California, 94111", 
                                      as_featureset=True, max_locations=1)
depot_geocoded_fs.sdf

,Loc_name,Status,Score,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,...,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,OBJECTID,SHAPE
0,World,M,99.71,"98 Pier 1, San Francisco, California, 94111","98 Pier 1, San Francisco, CA, 94111, USA",98 Pier 1,StreetAddress,,,"98 Pier 1, San Francisco, California, 94111",...,37.79752,-122.393818,37.79752,-122.394818,-122.392818,37.79652,37.79852,2,1,"{""type"": ""Point"", ""coordinates"": [-122.3938179..."


In [57]:
import json
with open("/arcgis/home/Routes_fset.json") as f:
    df1 = json.load(f)
routes_string = json.dumps(df1)    
routes_fset2 = FeatureSet.from_json(routes_string)
routes_fset2

<FeatureSet> 2 features

In [59]:
routes_string

'{"features": [{"attributes": {"StartDepotName": "Warehouse", "EndDepotName": "Warehouse", "EarliestStartTime": -2209132800000, "LatestStartTime": -2209125600000, "Capacities": "6", "CostPerUnitTime": 20, "MaxOrderCount": 6, "MaxTotalTime": 150, "AssignmentRule": 1, "Name": "Route1"}}, {"attributes": {"StartDepotName": "Warehouse", "EndDepotName": "Warehouse", "EarliestStartTime": -2209132800000, "LatestStartTime": -2209125600000, "Capacities": "6", "CostPerUnitTime": 20, "MaxOrderCount": 6, "MaxTotalTime": 150, "AssignmentRule": 1, "Name": "Route2"}}]}'

In [60]:

%%time
today = datetime.datetime.now()
from arcgis.network.analysis import solve_vehicle_routing_problem
results = solve_vehicle_routing_problem(orders= orders_fset,
                                        depots = depots_fset,
                                        routes = routes_fset, 
                                        save_route_data='true',
                                        populate_directions='true',
                                        travel_mode="Driving Time",
                                        default_date=today)

print('Analysis succeeded? {}'.format(results.solve_succeeded))

WARNING 030194: Data values longer than 500 characters for field [Routes:StartDepotName] are truncated.
WARNING 030194: Data values longer than 500 characters for field [Routes:EndDepotName] are truncated.
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Through Traffic Prohibited").


Analysis succeeded? True
CPU times: user 120 ms, sys: 5.47 ms, total: 126 ms
Wall time: 9.52 s


In [61]:
# Display the output routes in a pandas dataframe.
out_routes_df = results.out_routes.sdf
out_routes_df[['Name','OrderCount','StartTime','EndTime','TotalCost','TotalDistance','TotalTime','TotalTravelTime','StartTimeUTC','EndTimeUTC']]

,Name,OrderCount,StartTime,EndTime,TotalCost,TotalDistance,TotalTime,TotalTravelTime,StartTimeUTC,EndTimeUTC
0,Route1,4,2021-10-23 08:00:00,2021-10-23 09:57:27.687000036,2349.229061,60.709163,117.461453,117.461453,2021-10-23 15:00:00,2021-10-23 16:57:27.687000036
1,Route2,3,2021-10-23 08:00:00,2021-10-23 09:03:15.569000006,1265.189787,32.775232,63.259489,63.259489,2021-10-23 15:00:00,2021-10-23 16:03:15.569000006
2,Route3,3,2021-10-23 08:00:00,2021-10-23 09:27:35.410000086,1751.803183,40.886131,87.590159,87.590159,2021-10-23 15:00:00,2021-10-23 16:27:35.410000086


In [62]:

out_stops_df = results.out_stops.sdf
out_stops_df[['Name','RouteName','Sequence','ArriveTime','DepartTime']].sort_values(by=['RouteName',
                                                                                       'Sequence'])

,Name,RouteName,Sequence,ArriveTime,DepartTime
10,Warehouse,Route1,1,2021-10-23 08:00:00.000000000,2021-10-23 08:00:00.000000000
5,Location 6,Route1,2,2021-10-23 08:28:17.188999891,2021-10-23 08:28:17.188999891
6,Location 7,Route1,3,2021-10-23 08:39:31.591000080,2021-10-23 08:39:31.591000080
9,Location 10,Route1,4,2021-10-23 09:05:06.805999994,2021-10-23 09:05:06.805999994
7,Location 8,Route1,5,2021-10-23 09:28:19.418999910,2021-10-23 09:28:19.418999910
11,Warehouse,Route1,6,2021-10-23 09:57:27.687000036,2021-10-23 09:57:27.687000036
12,Warehouse,Route2,1,2021-10-23 08:00:00.000000000,2021-10-23 08:00:00.000000000
1,Location 2,Route2,2,2021-10-23 08:07:41.457999945,2021-10-23 08:07:41.457999945
3,Location 4,Route2,3,2021-10-23 08:28:20.938999891,2021-10-23 08:28:20.938999891
4,Location 5,Route2,4,2021-10-23 08:44:12.849999905,2021-10-23 08:44:12.849999905


In [63]:
# Create a map instance to visualize outputs in map
map_view_outputs = my_gis.map('San Fransisco, United States', zoomlevel=10)
map_view_outputs

MapView(layout=Layout(height='400px', width='100%'))

In [67]:
#Visusalize the inputsn with different symbols
map_view_outputs.draw(orders_fset, symbol={"type": "esriSMS",
                                           "style": "esriSMSCircle",
                                           "color": [76,115,0,255],"size": 8})
map_view_outputs.draw(depots_fset, symbol={"type": "esriSMS",
                                           "style": "esriSMSSquare",
                                           "color": [255,115,0,255], "size": 10})

#Visualize the first route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[0])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [0,100,240,255],"size":10})

#Visualize the second route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[1])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [255,0,0,255],"size":10})

#Visualize the second route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[2])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, 
                      symbol={"type": "esriSLS",
                              "style": "esriSLSSolid",
                              "color": [100,0,0,200],"size":10})

In [73]:
route_data = results.out_route_data.download('.')
route_data

'./_ags_rd2eb33df164e64a91b4be341c9df643c1.zip'

In [74]:

route_data_item = my_gis.content.add({"type": "File Geodatabase"}, route_data)
route_data_item

<Item title:"_ags_rd2eb33df164e64a91b4be341c9df643c1" type:File Geodatabase owner:Rudra_CSRE_User>

In [75]:
route_layers = arcgis.features.analysis.create_route_layers(route_data_item, 
                                                            delete_route_data_item=True)
for route_layer in route_layers:
    route_layer.share(org=True)
    display(route_layer.homepage)
    display(route_layer)

'https://www.arcgis.com/home/item.html?id=ab70f2f5a76842cba5a00c9ad4e5ebe5'

<Item title:"Route1" type:Feature Collection owner:Rudra_CSRE_User>

'https://www.arcgis.com/home/item.html?id=32b28ec8cc2341f8964bb2390bad024a'

<Item title:"Route2" type:Feature Collection owner:Rudra_CSRE_User>

'https://www.arcgis.com/home/item.html?id=0edeed191688462b8a60dab10bfb19fe'

<Item title:"Route3" type:Feature Collection owner:Rudra_CSRE_User>